In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308877  [    0/60000]
loss: 2.302120  [ 6400/60000]
loss: 2.294178  [12800/60000]
loss: 2.286149  [19200/60000]
loss: 2.294850  [25600/60000]
loss: 2.277118  [32000/60000]
loss: 2.289274  [38400/60000]
loss: 2.281783  [44800/60000]
loss: 2.282813  [51200/60000]
loss: 2.254593  [57600/60000]
Test Error: 
 Accuracy: 20.1%, Avg loss: 2.263614 

Epoch 2
-------------------------------
loss: 2.275988  [    0/60000]
loss: 2.281946  [ 6400/60000]
loss: 2.255258  [12800/60000]
loss: 2.248026  [19200/60000]
loss: 2.274374  [25600/60000]
loss: 2.231627  [32000/60000]
loss: 2.265511  [38400/60000]
loss: 2.248495  [44800/60000]
loss: 2.255082  [51200/60000]
loss: 2.205910  [57600/60000]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.219608 

Epoch 3
-------------------------------
loss: 2.245770  [    0/60000]
loss: 2.253527  [ 6400/60000]
loss: 2.205147  [12800/60000]
loss: 2.198019  [19200/60000]
loss: 2.243914  [25600/60000]
loss: 2.175918  [32000/600